In [1]:
from RDMM.evaluation_framework import EvaluationFramework
from pathlib import Path
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', 100)
from collections import namedtuple
from tqdm.notebook import tqdm
from RDMM.utils import *

## Setting up the paths where the results of the experiments are stored

This path should be the same as the one used in `run_experiments.py`.

In [2]:
date_path=Path('06_04')
pre_folder = Path.home()/Path('experiments')

In [3]:
folder= pre_folder/date_path
frame = EvaluationFramework(folder)

### Below specify what kind of experiments have been conducted

Currently we have selected the "regression" (regression) model class and the exhaustive approach.

In [4]:
#model_class=Path('regression')
#model_class=Path('transition')
model_class=Path('regression')

#strategy = "mine_pair"
strategy = "exhaustive"

In [5]:
if str(model_class)=="regression":
    num_dataframes = 10
    num_parameters = 36
    num_quality_pairs = 9
    experiment_name = strategy+"_results"
elif str(model_class)=="cov":
    num_dataframes = 10
    num_parameters = 36
    num_quality_pairs = 1
    experiment_name = strategy+"5_results"

In [6]:
def evaluate_results(self, prefix, experiment_name, n_results, show_progress=None, allow_omit=False):
    the_range=range(n_results)
    if not show_progress is None:
        the_range=show_progress(the_range)
    total_runtime={}
    mine_runtime={}
    for n in the_range:
        tpl = self.load_dataset(prefix/Path(experiment_name), n)
        fixed_params = (tpl.alpha, tpl.beta,tpl.gamma, tpl.ex_qf_name, tpl.sim_qf_name)
        mine_runtime[fixed_params] = (tpl.mine_runtime,)
        total_runtime[fixed_params] = (tpl.total_runtime,)
    mine_runtime=self.dict_to_df(mine_runtime)
    mine_runtime.columns=["mine_runtime"]
    total_runtime=self.dict_to_df(total_runtime)
    total_runtime.columns=["total_runtime"]
    return total_runtime, mine_runtime

In [7]:
total_experiments=num_dataframes * num_parameters * num_quality_pairs
result_frames  = evaluate_results(frame, model_class, experiment_name, total_experiments, tqdm, allow_omit=False)

  0%|          | 0/3240 [00:00<?, ?it/s]

In [8]:
abbreviations={'LikelihoodExceptionality_transition':'Like',
'LikelihoodSimilarity':'Like',
'TotalVariationSimilarity': 'var',
 'TotalVariationExceptionality':'var',
 'ParameterDiff':'par',
 'ParameterDiff_sim':'par',
 'Cooks_sim':'Cooks',
 'Like_sim':'Like',
 'Log_sim':'Log',
 'par_sim':'par',
  'Norm 1':'Norm1'}

In [9]:
def unstack_index(df, fun):
    new_df = df.groupby(['ex','sim']).agg(fun)
    #new_df = new_df.reorder_levels(['ex','sim'])
    return new_df

In [10]:
df_total = unstack_index(result_frames[0], ['mean','std']).sort_index()
df_mine = unstack_index(result_frames[1], ['mean','std']).sort_index()

In [11]:
df_total

total_runtime           
                         mean        std
ex    sim                               
Cooks Cooks_sim     97.772485  10.471489
      Like_sim      60.630669   7.746077
      par_sim       63.357430   7.419849
Like  Cooks_sim    108.396113  16.779303
      Like_sim      74.989559  13.029288
      par_sim       72.192890  12.676707
par   Cooks_sim     98.906005  10.656750
      Like_sim      61.195644   6.315275
      par_sim       64.407764   6.305138

In [12]:
df_mine.columns=df_mine.columns.droplevel()

In [13]:
df_total.columns=df_total.columns.droplevel()

In [14]:
(df_total-df_mine).groupby(['sim']).agg('mean')

,mean,std
sim,,
Cooks_sim,98.379732,11.986317
Like_sim,62.452288,8.458218
par_sim,63.961637,8.260196


In [15]:
(df_total).groupby(['sim']).agg('mean')

,mean,std
sim,,
Cooks_sim,101.691534,12.635847
Like_sim,65.605291,9.030213
par_sim,66.652695,8.800565


In [16]:
df_mine = result_frames[1].groupby(['sim']).agg(['mean','std']).sort_index()
df_total2 = result_frames[0].groupby(['sim']).agg(['mean','std']).sort_index()

In [17]:
df_total2

total_runtime           
                   mean        std
sim                               
Cooks_sim    101.691534  13.711726
Like_sim      65.605291  11.519534
par_sim       66.652695   9.962591